In [ ]:
#|default_exp hits.models

In [ ]:
#|export
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# |export
def tcsConv(n_in, n_out, k_size=3, stride=1, dilation=1, norm=True, act=None, pointwise=True):
    '''Time-Channel Separable Convolution'''
    if not pointwise: conv = nn.Sequential(nn.Conv1d(n_in, n_out, k_size,stride=stride, dilation=dilation))
    else:
        depthwise = nn.Conv1d(n_in, n_in, k_size, groups=n_in, dilation=dilation, padding='same')
        pointwise = nn.Conv1d(n_in, n_out, 1)
        conv = nn.Sequential(depthwise, pointwise)
    if norm: conv.append(nn.BatchNorm1d(n_out))
    if act: conv.append(act)
    return conv


class TCSBlock(nn.Module):
    '''Block from http://arxiv.org/abs/2004.08531'''

    def __init__(self, n_in, n_out, k_size):
        super().__init__()
        self.n_in = n_in
        self.n_out = n_out
        self.k_size = k_size
        self.tcs_conv = tcsConv(self.n_in, self.n_out, self.k_size)
        self.dropout = nn.Dropout()

    def forward(self, x, residual=None):
        x = self.tcs_conv(x)
        if residual is not None: x = x + residual
        x = F.relu(x)
        x = self.dropout(x)
        return x


class MainBlock(nn.Module):
    def __init__(self, n_in, n_out, k_size, R=1):
        super().__init__()
        self.n_in = n_in
        self.n_out = n_out
        self.k_size = k_size
        self.R = R

        self.inner_blocks = nn.ModuleList([TCSBlock(self.n_in, self.n_out, self.k_size)])
        for _ in range(R-1):
            self.inner_blocks.append(
                TCSBlock(self.n_out, self.n_out, self.k_size))
        self.residual = tcsConv(self.n_in, self.n_out,
                                1, act=None, pointwise=False)

    def forward(self, x):
        res = self.residual(x)
        for i, block in enumerate(self.inner_blocks):
            if (i+1) == len(self.inner_blocks):
                x = block(x, res)
            else:
                x = block(x)
        return x


class MatchBoxNetwork(nn.Module):
    def __init__(self, n_in, C, B=3, R=2, n_classes=1, k_init=11):
        super().__init__()
        k_sizes = [k*2+k_init for k in range(1, B)]
        self.prologue = tcsConv(n_in, 128, k_init, stride=2,
                                act=nn.ReLU(), pointwise=False)

        self.blocks = nn.ModuleList([MainBlock(128, C, k_sizes[0], R)])
        for k in k_sizes[1:]:
            self.blocks.append(MainBlock(C, C, k, R))

        self.epilogue = nn.Sequential(
            tcsConv(C, 128, k_init, dilation=2, pointwise=False),
            tcsConv(128, 128, 1),
            tcsConv(128, n_classes, 1),
            nn.AdaptiveAvgPool1d(1))
    
    def forward(self, x):
        z = self.prologue(x)
        for block in self.blocks:
            z = block(z)
        z = self.epilogue(z).squeeze()
        return z